In [35]:
from mpl_toolkits.basemap import Basemap
import MITgcmutils as mit
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
from mitgcm_python.plot_latlon import latlon_plot, shade_land,plot_vel
from mitgcm_python.grid import Grid
from datetime import datetime
import xarray as xr
import geopandas as gpd
from skimage import io
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.lines as mlines
from codecs import encode
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

In [36]:
# Fungsi 
def getclosest_ij(lats,lons,latpt,lonpt):    
    """Function to find the index of the closest point to a certain lon/lat value."""
    dist_lat = (lats-latpt)**2          # find squared distance of every point on grid
    dist_lon = (lons-lonpt)**2
    minindex_lat = dist_lat.argmin()    # 1D index of minimum dist_sq element
    minindex_lon = dist_lon.argmin()
    return minindex_lat, minindex_lon
def act_to_list(act_file):
    """
    Converts act file to a list.
    Args:
        act_file (str): Full path to act file.
    Returns:
        colors (list): List of colors in hexcode string format.
    """
    # Read binary data
    with open(act_file, 'rb') as act:
        raw_data = act.read()
    # Convert it to hexadecimal values
    hex_data = encode(raw_data, 'hex')
    # Decode colors from hex to string and split it by 6 (because colors are #1c1c1c)
    colors = [hex_data[i:i+6].decode() for i in range(0, (len(hex_data)-8), 6)]
    # Add # to each item and filter empty items if there is a corrupted total_colors_count bit
    colors = ['#'+i for i in colors if len(i)]
    return colors
def load_idl_cmap(fn):
    """
    Loads colormaps from the IDL library.
    Args:
        fn (str): Name of the desired colormap. E.g. 'Table22'
    Returns:
        cmap (obj): Matplotlib LinearSegmentedColormap object.
    """
    path = './colortables/idl/'
    # load idl colormaps
    df = pd.read_csv(path + fn + '.txt', header=None, delim_whitespace=True)
    nn = df.shape[0]
    cmap_list = [[df.values[i, 0] / 255., df.values[i, 1] / 255., df.values[i, 2] / 255.] for i in range(nn)]
    cmap = LinearSegmentedColormap.from_list('cmap', cmap_list, len(cmap_list))
    return cmap
def load_panoply_cmap(fn, reverse=False, remove_black=False):
    """
    Loads colormaps from the Panoply library (https://www.giss.nasa.gov/tools/panoply/colorbars/).
    Args:
        fn (str): Name of the desired colormap. E.g. 'NEO_bright_temp'
    Returns:
        cmap (obj): Matplotlib LinearSegmentedColormap instance.
    """
    
    panoply_path = 'F:/Coding/hdf5 folder/hdf-project/data/karimun jawa/'
    cmaplist_hex = act_to_list(panoply_path + fn + '.act')
    cmaplist = [mpl.colors.hex2color(cmhex) for cmhex in cmaplist_hex]
    if reverse:
        cmaplist = cmaplist[::-1]
    if remove_black:
        # remove black
        cmaplist.remove((0., 0., 0.))
    cmap = mpl.colors.LinearSegmentedColormap.from_list('cmap', cmaplist, len(cmaplist))
    return cmap


In [37]:
#Setting Colormap yang akan digunakan
cmap_iw = load_panoply_cmap("NEO_bright_temp")

In [38]:
#setting waktu, iterasi dan tanggal pada data
nIter=79200 #802800 #84000 #345600
layer=1
tracername='T' #'DIC_pCO2tave' #'Salinity' #'Temperature' #'Elevation' #'Salinity' #'Velocity' #'temp'
stride  = 15            # plot every skip vectors in each direction
deltaT=60

epoch_time = datetime(2021,11, 1)
sec_time=nIter*deltaT
total_sec=sec_time+epoch_time.timestamp()
cur_time  = datetime.fromtimestamp(total_sec).strftime("%d %B, %Y")# %H:%M%p')
print("datetime is:", cur_time)

datetime is: 26 December, 2021


In [39]:
# data batimetri
filename='F:/Coding/hdf5 folder/hdf-project/data/karimun jawa/make_topo/make_topo/SRTM15Plus_Karimunjawa.nc'

domain=[108.5, 112.5, -7.25, -4.5]  #Jepara
outdir='./'

In [40]:
#Dictionary dari variabel yang akan di plot
var_dict=dict({"PTRACER01":{ "var": "DIC",
                "units": "mol/m$^3$",
                "vmin":1.85 ,
                "vmax":1.95 ,
                "dimension":"3D" ,                             
               },
               "PTRACER02":{ "var": "Alkalinity",
                "units": "mol/m$^3$",
                "vmin":2.2 ,
                "vmax":2.3 ,
                "dimension":"3D" , 
               },
                "PTRACER03":{ "var": "PO4",
                "units": "mol/m$^3$",
                "vmin":0.0 ,
                "vmax":0.0002 ,
                "dimension":"3D" ,                              
               },
               "PTRACER04":{ "var": "DOP",
                "units": "mol/m$^3$",
                "vmin":0.0 ,
                "vmax":1e-4 ,
                "dimension":"3D" ,          
               },
               "PTRACER05":{ "var": "O2",
                "units": "mol/m$^3$",
                "vmin":0.19 ,
                "vmax":0.21 ,
                "dimension":"3D" ,   
               },                  
               "T":{ "var": "Temperature",
                "units": "${^o}$C",
                "vmin":27 ,
                "vmax":31 ,
                "dimension":"3D" ,                            
               },
                "S":{ "var": "Salinity",
                "units": "psu",
                "vmin":32 ,
                "vmax":33 ,
                "dimension":"3D" ,        
               },
                "DIC_pCO2tave":{ "var": "pCO2",
                "units": "$/mu$atm",
                "vmin":350 ,
                "vmax":500 ,
                "dimension":"2D" ,        
               },               
               })

In [41]:
#Setting Iterasi File MITgcm
striter='{:010d}'.format(nIter)
grid_path='F:/Coding/hdf5 folder/hdf-project/data/karimun jawa/'
indgrid=Grid(grid_path)

In [42]:
x=mit.rdmds(grid_path+'XG'); y=mit.rdmds(grid_path+'YG');
z=mit.rdmds(grid_path+'RC')
depth =z[layer-1]*-1.
if depth== 0.5:
    level='surface'
else:
    level=str(depth).lstrip('[').rstrip(']')+ 'm'

tracer = mit.rdmds(grid_path+tracername,nIter)
#salt = mit.rdmds('S',nIter)
uVel = mit.rdmds(grid_path+'U',nIter)
vVel = mit.rdmds(grid_path+'V',nIter)
#maskInC=mit.rdmds(grid_path+'maskInC')
hFacW=mit.rdmds(grid_path+'hFacW')
hFacS=mit.rdmds(grid_path+'hFacS')
hFacC=mit.rdmds(grid_path+'hFacC')

hFacW=hFacW+1
hFacW[hFacW==2]=0
hFacS=hFacS+1
hFacS[hFacS==2]=0
hFacC=hFacC+1
hFacC[hFacC==2]=0
u_vec=np.ma.MaskedArray(uVel, hFacW)
v_vec=np.ma.MaskedArray(vVel, hFacS)
if list(var_dict.get(tracername).values())[4] == '3D' :
    tracer=np.ma.MaskedArray(tracer, hFacC)
    t_plot=tracer[layer-1,:,:]
else:
    t_plot=np.ma.MaskedArray(tracer, hFacC[0,:,:])
#u_vec=np.ma.MaskedArray(uVel, hFacC)
#v_vec=np.ma.MaskedArray(vVel, hFacC)
u_plot=u_vec[layer-1]
v_plot=v_vec[layer-1]

##Units conversion or scaling
if list(var_dict.get(tracername).values())[0] == 'pCO2' :
    t_plot=t_plot*1e6

mag = np.sqrt(u_plot**2 + v_plot**2)


iy_min, ix_min = getclosest_ij(y[:,0], x[0,:], domain[2], domain[0])
iy_max, ix_max = getclosest_ij(y[:,0], x[0,:], domain[3], domain[1])


In [43]:
#Membut meshgrid menggunakan Datarray library Xarray
da = xr.DataArray(
      data=t_plot,
      dims=["y", "x"],
      coords=dict(
        x_length=(["y", "x"], x),
        y_length=(["y", "x"], y),
      ),
      attrs=dict(
        description="Tracer",
        units="read dict",
      ),
   )

tr01min=xr.DataArray(list(var_dict.get(tracername).values())[2]).astype(dtype='float64') #da.min() #1.7 # 250
tr01max=xr.DataArray(list(var_dict.get(tracername).values())[3]) #da.max() #2.4 #1000
interval=(tr01max-tr01min)/5
print('debug range ', da.min(), '-', da.max())

debug range  <xarray.DataArray ()> Size: 8B
array(28.89722443) - <xarray.DataArray ()> Size: 8B
array(30.82801247)


In [44]:
#Data Coastline
coastlines = gpd.read_file('F:/Coding/hdf5 folder/hdf-project/data/karimun jawa/make_topo/make_topo/coastlines-split-4326/lines.shp', bbox=(domain[0], domain[3], domain[1],domain[2]))


In [45]:
proj4326 = 'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?\
version=1.3.0&service=WMS&request=GetMap&\
format=image/jpeg&STYLE=default&bbox={},{},{},{}&CRS=EPSG:4326&\
HEIGHT=512&WIDTH=512&layers=BlueMarble_NextGeneration'.format(domain[2],domain[0],domain[3],domain[1])

In [46]:
# Open BATNAS
ds=xr.open_dataset(filename)
# Request image from WMS (web_map_service)
img = io.imread(proj4326)

#-----
transform=ccrs.PlateCarree()
subplot_kws=dict(projection=ccrs.PlateCarree(),
                 facecolor='grey')
size = (14, 10)

In [47]:
#plot kondisi awal
#-----------------
# Defining the figure
fig = plt.figure(figsize=size, facecolor='w', edgecolor='k')
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.set_xlim([domain[0], domain[1]])
ax.set_ylim([domain[2],domain[3]])

#fig, ax = plt.subplots()
p = da.plot(x='x_length', y='y_length',vmin=tr01min, vmax=tr01max,cmap=cmap_iw,
                subplot_kws=subplot_kws,
               transform=ccrs.PlateCarree(),               
                  add_labels=False,
                  add_colorbar=False)

cbaxes = fig.add_axes([0.85, 0.11, 0.03, 0.77])
cbar=plt.colorbar(p,cax=cbaxes, orientation='vertical')
cbar.set_label(list(var_dict.get(tracername).values())[1],
               rotation=270,fontsize=18, labelpad=20)

cbar.ax.set_yticklabels(["{:.2f}".format(i) for i in cbar.get_ticks()], size=18)

cintervals =  np.arange(tr01min,tr01max, interval)

#cbar.set_ticks(cintervals)

CL = ax.contour(x,y,da,cintervals,transform=ccrs.PlateCarree(),linewidths=1,colors='silver', linestyles='--', origin='lower')
ax.clabel(CL, inline_spacing=0.5, fontsize=18,colors='white', fmt='%.2f')

ax = coastlines.plot(color='white', ax=ax)
im=ax.imshow(img, extent = (domain[0], domain[1], domain[2], domain[3]), origin = 'upper')

q=ax.quiver(x[::stride,::stride],y[::stride,::stride], u_plot[::stride,::stride], v_plot[::stride,::stride], scale=20, headwidth=5, headlength=5,headaxislength=2.8)
qk=ax.quiverkey(q, 0.9, 0.05, 1, r'1 m/s' , fontproperties={'size': '14', },color='white', labelcolor='white',edgecolor="white")

# draw parallels/meridiens and write labels
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.1, linestyle='--')

# adjust labels to taste
gl.top_labels = False
gl.right_labels = False
cintervals = np.arange(domain[0], domain[1], 1)
gl.xlocator = mticker.FixedLocator(cintervals)
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 24, 'color': 'black'}
gl.ylabel_style = {'size': 24, 'color': 'black'}
ax.text(109.75, -7.15, r'Central Java', fontsize=36, color='white')
ax.set_title(list(var_dict.get(tracername).values())[0] +' at '+ level + ' on '+ cur_time,fontsize=24)

plt.show()

C:\Users\Feel\AppData\Local\Temp\ipykernel_13764\4012499451.py:21: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  cbar.ax.set_yticklabels(["{:.2f}".format(i) for i in cbar.get_ticks()], size=18)
